In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands



In [2]:
def fingertip(hand_landmarks,image):
    val = hand_landmarks
    val2 = str(val)
    st = val2.split("landmark")
    arr = st[9].split("\n")
    h_loc_y = float(arr[2].split(" ")[3])
    h_loc_x = float(arr[1].split(" ")[3])
    h_loc_z = float(arr[3].split(" ")[3])
    h_loc_y = float(h_loc_y*480)
    h_loc_x = float(h_loc_x*640)
    image = cv2.flip(image, 1)
    image = cv2.flip(image, 1)
    return image

In [3]:
def tip(hand_landmarks,image, a):
    val = hand_landmarks
    val2 = str(val)
    st = val2.split("landmark")
    arr = st[a].split("\n")
    h_loc_y = float(arr[2].split(" ")[3])
    h_loc_x = float(arr[1].split(" ")[3])
    h_loc_z = float(arr[3].split(" ")[3])
    h_loc_y = float(h_loc_y*480)
    h_loc_x = float(h_loc_x*640)
    co = np.array((h_loc_x, h_loc_y))
    return co

In [4]:
def split(arr, cond):
      return [arr[cond], arr[~cond]]

In [5]:
def array_split(newarr):
    index = np.where(newarr=='b')
    index = np.array(index)
    index = index.astype(int)
    arrays = np.split(newarr,index[0])
    return arrays

In [6]:
def text_wrapper(image, a, b, flag, x1, y1, arr):
    euc = np.linalg.norm(a-b)
    image = cv2.flip(image,1)
    if euc <= 60:
        
        image = cv2.putText(image, str("Write"), (50,80), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0),2,cv2.LINE_AA)
        if flag == 0:
            x1,y1 = a[0],a[1]
            arr.append((int(x1),int(y1)))
            nparr = np.array(arr)
        else:
            x2,y2 = a[0],a[1]
            arr.append((int(x2),int(y2)))

            nparr = np.array(arr)
            x1,y1 = x2,y2
        flag = 1
        
    else:
        flag = 0
        arr.append("b")
        nparr = np.array(arr)
    image = cv2.flip(image,1)
    return image, flag, x1, y1, nparr

In [7]:
cap = cv2.VideoCapture(0)
flag = 0
x1,y1 = 0,0
arr = []
nparr = np.array(arr)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        #mp_drawing.draw_landmarks(image,hand_landmarks,mp_hands.HAND_CONNECTIONS,mp_drawing_styles.get_default_hand_landmarks_style(),mp_drawing_styles.get_default_hand_connections_style())
        
        ind = tip(hand_landmarks, image, 9)
        th = tip(hand_landmarks, image, 5)
        image, flag1, x1, y1, nparr1 = text_wrapper(image, ind,th, flag, x1,y1,arr)
        nparr = array_split(nparr1)
        flag = flag1
    #if cv2.waitKey(33) & 0xFF == ord('d'):
        #nparr = np.empty[1,1]
 
    # Flip the image horizontally for a selfie-view display.
    for i in nparr:
        nparr_s = i[i!='b']
        for point1, point2 in zip(nparr_s, nparr_s[1:]): 
            cv2.line(image, point1, point2, [0, 255, 0], 2)
            

    image = cv2.resize(image, (720, 480))
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    #if cv2.waitKey(5) & 0xFF == 27:
     # break
    pressedKey = cv2.waitKey(5) & 0xFF
    if pressedKey == ord('q'):
        break
    elif pressedKey == ord('d'):
        arr = []
        image, flag1, x1, y1, nparr1 = text_wrapper(image, ind,th, flag, x1,y1,arr)
        nparr = nparr1

    
cap.release()